In [2]:
import geopandas as gpd

In [9]:
#Access the urban concentrations boundaries file
areasurbsrc = r'_raw_data/IBGE_Areas Urbanas/IBGE_AreasUrbanas_Limites.shp'
areasurb = gpd.read_file(areasurbsrc)
areasurb = areasurb.to_crs("EPSG:4326")

In [4]:
#Declare a variable for a subset of urban concentrations in each state
ceara = areasurb[areasurb['cd_concurb'].str.startswith('23')]
rondonia = areasurb[areasurb['cd_concurb'].str.startswith('11')]
amazonas = areasurb[areasurb['cd_concurb'].str.startswith('13')]
roraima = areasurb[areasurb['cd_concurb'].str.startswith('14')]
para = areasurb[areasurb['cd_concurb'].str.startswith('15')]
amapa = areasurb[areasurb['cd_concurb'].str.startswith('16')]
tocantins = areasurb[areasurb['cd_concurb'].str.startswith('17')]
maranhao = areasurb[areasurb['cd_concurb'].str.startswith('21')]
piaui = areasurb[areasurb['cd_concurb'].str.startswith('22')]
rn = areasurb[areasurb['cd_concurb'].str.startswith('24')]
paraiba = areasurb[areasurb['cd_concurb'].str.startswith('25')]
pernambuco = areasurb[areasurb['cd_concurb'].str.startswith('26')]
alagoas = areasurb[areasurb['cd_concurb'].str.startswith('27')]
sergipe = areasurb[areasurb['cd_concurb'].str.startswith('28')]
bahia = areasurb[areasurb['cd_concurb'].str.startswith('29')]
minas = areasurb[areasurb['cd_concurb'].str.startswith('31')]
es = areasurb[areasurb['cd_concurb'].str.startswith('32')]
rio = areasurb[areasurb['cd_concurb'].str.startswith('33')]
parana = areasurb[areasurb['cd_concurb'].str.startswith('41')]
sampa = areasurb[areasurb['cd_concurb'].str.startswith('35')]
santa = areasurb[areasurb['cd_concurb'].str.startswith('42')]
rs = areasurb[areasurb['cd_concurb'].str.startswith('43')]
matogrossodosul = areasurb[areasurb['cd_concurb'].str.startswith('50')]
matogrosso = areasurb[areasurb['cd_concurb'].str.startswith('51')]
goiania = areasurb[areasurb['cd_concurb'].str.startswith('52')]
df = areasurb[areasurb['cd_concurb'].str.startswith('53')]
acre = areasurb[areasurb['cd_concurb'].str.startswith('12')]

brasil = [ceara,rondonia,amazonas,roraima,para,amapa,tocantins,maranhao,piaui,rn,paraiba,pernambuco,alagoas,sergipe,
         bahia,minas,es,rio,parana,sampa,santa,rs,matogrossodosul,matogrosso,goiania,df,acre]

In [107]:
#Initializing Google Earth Engine
import ee
import geehydro
ee.Initialize()

#List of land use classes from mabiomas
fromList = [24,3,4,5,49,11,12,32,29,13,15,18,9,21,23,30,25,33,31,27];
#We're only interested in class 24, so everything else should become 0
toList =   [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0];


for state in range(0,len(brasil)):
    for year in range(1993,2021):
        #Defining the area of interest from the parcel's bounding box
        envelope = brasil[state].geometry.total_bounds
        aoi=ee.Geometry.Rectangle(envelope.tolist())
        #Accessing the latest collection (6) and filter to the year and area of interest
        Mapbiomas = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1').select('classification_'+str(year)).clip(aoi)
        Mapbiomas = Mapbiomas.remap(fromList, toList, 0,'classification_'+str(year))

        ee.batch.Export.image.toDrive(
            image = Mapbiomas.clip(aoi),
            folder = 'Mapbiomas_urban',
            fileNamePrefix = str(state)+'_'+str(year),
            scale=30,
            maxPixels = 45000000000
        ).start()

In [3]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
from shapely.geometry import mapping
import matplotlib.pyplot as plt
import glob
import os

In [4]:
estadossrc = r'_raw_data/bra_adm_ibge_2020_shp/bra_admbnda_adm1_ibge_2020.shp'
estados = gpd.read_file(estadossrc)
estados = estados.set_crs('EPSG:4326')

In [5]:
pares={0:'Ceará',1:'Rondônia',2:'Amazonas',3:'Roraima',4:'Pará',5:'Amapá',6:'Tocantins',7:'Maranhão',8:'Piauí',9:'Rio Grande do Norte',10:'Paraíba',11:'Pernambuco',12:'Alagoas',13:'Sergipe',
         14:'Bahia',15:'Minas Gerais',16:'Espírito Santo',17:'Rio de Janeiro',18:'Paraná',19:'São Paulo',20:'Santa Catarina',21:'Rio Grande do Sul',22:'Mato Grosso do Sul',23:'Mato Grosso',24:'Goiânia',25:'Distrito Federal',26:'Acre'}

In [14]:
#Import the libraries
import pprint
import rasterio
from rasterio import features
from shapely.geometry import shape

#Access the folder where you saved the exports from Google Earth Engine
dirpath = r"_raw_data/expansao_imagens"

tudo = gpd.GeoDataFrame(columns=['lulc', 'year', 'src', 'geometry'], geometry='geometry', crs="EPSG:4326")


for year in range(1995,2021):
    mastergdf = gpd.GeoDataFrame(columns=['lulc', 'year', 'src', 'geometry'], geometry='geometry', crs="EPSG:4326")
    # Make a search criteria to select the raster files
    search_criteria = "*"+str(year)+".tif"
    q = os.path.join(dirpath, search_criteria)
    dem_fps = glob.glob(q)
    #print(dem_fps)
    for fp in dem_fps:
        #Open the MapBiomas export
        with rasterio.open(fp) as src:
            lulc = src.read(1)
            #Create a rasterio feature object by ingesting the land-cover raster
            shapes = features.shapes(lulc,transform=src.transform)

            #Create empty lists to store the values
            urban = []
            geometry = []

            #Populate the lists with the geometries and values from the feature object
            for shapedict, value in shapes:
                urban.append(value)
                geometry.append(shape(shapedict))

            #Create a geodataframe with the values from the lists
            gdf = gpd.GeoDataFrame(
                {'lulc': urban, 'year': year, 'src':fp, 'geometry': geometry },
                crs="EPSG:4326")

            #Filter the features that don't correspond to the agriculture category
            classes = [1]
            gdf = gdf[gdf["lulc"].isin(classes)]
            mastergdf = mastergdf.append(gdf)
            print(fp+" ok")

    mastergdf = gpd.overlay(mastergdf, areasurb, how='intersection')
    mastergdf.to_file(str(year)+".shp")
    


/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/24_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/25_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/2_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/3_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/8_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/9_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/10_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/11_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/23_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/22_1995.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/5_1995.tif ok
/Users/guilhermeiablonovski/Desktop/W

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/17_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/16_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/5_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/4_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/23_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/22_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/10_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/11_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/8_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/9_1998.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/2_1998.tif ok
/Users/guilhermeiablonovski/Desktop/W

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/4_2001.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/12_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/13_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/26_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/18_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/19_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/0_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/1_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/15_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/14_2002.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/21_2002.tif ok
/Users/guilhermeiablonovski/Desktop

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/20_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/3_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/2_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/25_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/24_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/11_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/10_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/9_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/8_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/4_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/5_2005.tif ok
/Users/guilhermeiablonovski/Desktop/WR

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/12_2008.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/13_2008.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/17_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/16_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/5_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/4_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/23_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/22_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/10_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/11_2009.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/8_2009.tif ok
/Users/guilhermeiablonovski/Desktop

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/15_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/14_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/11_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/10_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/9_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/8_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/3_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/2_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/25_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/24_2012.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/16_2012.tif ok
/Users/guilhermeiablonovski/Desktop/

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/23_2015.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/4_2015.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/5_2015.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/13_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/12_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/26_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/19_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/18_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/1_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/0_2016.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/14_2016.tif ok
/Users/guilhermeiablonovski/Desktop/

/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/8_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/9_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/10_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/11_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/14_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/15_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/20_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/21_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/6_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/7_2019.tif ok
/Users/guilhermeiablonovski/Desktop/WRI_Expansao urbana/expansao_imagens/13_2019.tif ok
/Users/guilhermeiablonovski/Desktop/

In [18]:
#Dissolve by concurb and replace the shapefile
dirpath = r"_raw_data/expansao_shapes"
search_criteria = "*.shp"
q = os.path.join(dirpath, search_criteria)
dem_fps = glob.glob(q)

for fp in dem_fps:
    gdf = gpd.read_file(fp)
    gdf = gdf.dissolve(by='cd_concurb')
    gdf.to_file(fp)



In [33]:
#Optionally create one single large file with all years
onion = gpd.read_file(r'_raw_data/expansao_shapes/1993.shp')
for year in range(1993,2020):
    currentsrc = r'_raw_data/expansao_shapes/'+str(year+1)+'.shp'
    previoussrc = r'_raw_data/expansao_shapes/'+str(year)+'.shp'
    current = gpd.read_file(currentsrc)
    previous = gpd.read_file(previoussrc)
    res_difference = current.overlay(previous, how='difference')
    onion = onion.append(res_difference)
onion.to_file('onion.shp')